###Задание по программированию: Опорные объекты

работать с методом опорных векторов (SVM)

находить наиболее важные объекты выборки

In [1]:
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving data-logistic.csv to data-logistic.csv


In [ ]:
df = pd.read_csv("svm-data.csv", header=None)

In [ ]:
df.head()

,0,1,2
0,0.0,0.70,0.29
1,1.0,0.23,0.55
2,0.0,0.72,0.42
3,0.0,0.98,0.68
4,0.0,0.48,0.39


In [ ]:
X_train = df[df.columns[1:3]]

In [ ]:
y_train = df[df.columns[0]]

In [ ]:
from sklearn.svm import SVC
#kernel='linear, support_ = индексы опорных векторов

In [ ]:
clr = SVC(kernel='linear', C=100000, random_state=241)

In [ ]:
clr.fit(X_train, y_train)

SVC(C=100000, kernel='linear', random_state=241)

In [ ]:
n_s = clr.support_

In [ ]:
' '.join([str(n + 1) for n in n_s])

'4 5 10'

###Задание по программированию: Анализ текстов

находить оптимальные параметры для метода опорных векторов

работать с текстовыми данными

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

In [ ]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [ ]:
X = newsgroups.data
y = newsgroups.target

вычисление TF-IDF

Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_train = vectorizer.fit_transform(X)

In [ ]:
X_train.shape

(1786, 28382)

In [ ]:
feature_mapping = vectorizer.get_feature_names()
feature_mapping[28380]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'zwork'

In [ ]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_train, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [ ]:
C_best = gs.best_params_['C']

In [ ]:
C_best

1.0

In [ ]:
gs.best_score_

0.9932804406678872

In [ ]:
clf.set_params(C = C_best).fit(X_train, y)

SVC(kernel='linear', random_state=241)

In [ ]:
absolute_data = abs(clf.coef_.toarray().reshape(-1)) # by absolute
absolute_data_sorted_desc = sorted(absolute_data, reverse=True)

In [ ]:
weight_indexes = []
for weight in absolute_data_sorted_desc[:10]:
    weight_indexes.append(absolute_data.tolist().index(weight))   

In [ ]:
weight_indexes 

[24019, 12871, 5088, 5093, 17802, 23673, 21850, 5776, 15606, 22936]

In [ ]:
words = [vectorizer.get_feature_names()[index] for index in weight_indexes]

print('%s' % (" ".join(sorted(words))))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


atheism atheists bible god keith moon religion sci sky space


###Логистическая регрессия


работать с логистической регрессией

реализовывать градиентный спуск для ее настройки

использовать регуляризацию

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

In [ ]:
df = pd.read_csv("data-logistic.csv", header=None)

In [ ]:
df.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [ ]:
X_train = df[df.columns[1:3]]

In [ ]:
y_train = df[df.columns[0]]

In [ ]:
import math

In [ ]:
X_train.head()

,1,2
0,-0.663827,-0.138526
1,1.994596,2.468025
2,-1.247395,0.749425
3,2.309374,1.899836
4,0.849143,2.407750


In [ ]:
def calculate_w1(x,y,w1,w2,k,C):
  w = 0
  l = len(y)
  for i in range(0,l):
    w += y[i] * x[1][i] * (1 - 1 / (1 + np.exp(-y[i]*(w1*x[1][i] + w2*x[2][i]))))
  return w1 + k / l * w - k * C * w1

In [ ]:
def calculate_w2(x,y,w1,w2,k,C):
  w = 0
  l = len(y)
  for i in range(0,l):
    w += y[i] * x[2][i] * (1 - 1 / (1 + np.exp(-y[i]*(w1*x[1][i] + w2*x[2][i]))))
  return w2 + k / l * w - k * C * w2

In [ ]:
calculate_w2(X_train,y_train,0,0,0.1,10)

0.03245997127838869

In [ ]:
def gradient_descent(x,y,w1=0,w2=0,k=0.05,C=10,max_iter=10000, precision=0.00005):
  for i in range(max_iter):
    w1_prev, w2_prev = w1, w2
    w1 = calculate_w1(x,y,w1_prev,w2_prev,k,C)
    w2 = calculate_w2(x,y,w1_prev,w2_prev,k,C)
    if np.sqrt((w1_prev - w1) ** 2 + (w2_prev - w2) ** 2) <= precision:
            break
  return w1, w2

In [ ]:
w1, w2 = gradient_descent(X_train,y_train, C=0, w1=0.3, w2=0.05)

In [ ]:
w1, w2

(0.29114485078068747, 0.08890463342114549)

In [ ]:
w1_l2, w2_l2 = gradient_descent(X_train,y_train, w1=0.3, w2=0.05)

In [ ]:
w1_l2, w2_l2 

(0.02858258348365214, 0.024761725388058667)

In [ ]:
len(X_train)

205

In [ ]:
def sigmoid_a(x,w1,w2):
  y = []
  for i in range(len(x)):
    y.append(1 / (1 + np.exp(-w1*x[1][i]-w2*x[2][i])))
  return y


In [ ]:
y_train_gd = sigmoid_a(X_train,w1,w2)
y_train_gd_l2 = sigmoid_a(X_train,w1_l2,w2_l2)

In [ ]:
y_train_gd[:10]

[0.4495875539684014,
 0.6903760265686375,
 0.4292818725927742,
 0.6980260934291851,
 0.61529692718192,
 0.587029302009543,
 0.7019480495645131,
 0.5298602853258328,
 0.41977741050009937,
 0.596623233286887]

In [ ]:
y_train_gd_l2[:10]

[0.49440534414553206,
 0.5294880722631531,
 0.4957422791584185,
 0.5282183219065167,
 0.520963324295356,
 0.5062541034765359,
 0.5300732630971229,
 0.5086186375228098,
 0.4887412533431942,
 0.5175734745559635]

In [ ]:
len(y_train_gd)

205

In [ ]:
score = roc_auc_score(y_train,y_train_gd)

In [ ]:
score

0.9254285714285713

In [ ]:
score_l2 = roc_auc_score(y_train, y_train_gd_l2)

In [ ]:
score_l2

0.9364761904761904

###Задание по программированию: Метрики качества классификации

вычислять различные меры качества классификации: долю правильных ответов, точность, полноту, AUC-ROC и т.д.

сравнивать алгоритмы классификации при наличии ограничений на точность или полноту

sklearn.metrics


In [2]:
uploaded = files.upload()

Saving classification.csv to classification.csv


In [3]:
df = pd.read_csv("classification.csv")

In [4]:
df.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


In [5]:
df.shape

(200, 2)

In [7]:
TP = df[(df["pred"]==1) & (df["true"]==1)]

In [9]:
FP = df[(df["pred"]==1) & (df["true"]==0)]

In [10]:
FN = df[(df["pred"]==0) & (df["true"]==1)]

In [11]:
TN = df[(df["pred"]==0) & (df["true"]==0)]

In [12]:
len(TP), len(FP), len(FN), len(TN)

(43, 34, 59, 64)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [17]:
Accuracy = accuracy_score(df['true'],df['pred'])
Accuracy

0.535

In [18]:
Precision = precision_score(df['true'],df['pred'])
Precision

0.5584415584415584

In [19]:
Recall = recall_score(df['true'],df['pred'])
Recall

0.4215686274509804

In [20]:
F1 = f1_score(df['true'],df['pred'])
F1

0.48044692737430167

Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

для логистической регрессии — вероятность положительного класса (колонка score_logreg),

для SVM — отступ от разделяющей поверхности (колонка score_svm),

для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),

для решающего дерева — доля положительных объектов в листе (колонка score_tree).

In [54]:
import numpy as np

In [21]:
uploaded = files.upload()

Saving scores.csv to scores.csv


In [22]:
df_scores = pd.read_csv('scores.csv')

In [23]:
df_scores.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


In [24]:
from sklearn.metrics import roc_auc_score

In [27]:
roc_log = roc_auc_score(df_scores['true'],df_scores['score_logreg'])
roc_log

0.719187675070028

In [28]:
roc_svm = roc_auc_score(df_scores['true'],df_scores['score_svm'])
roc_svm

0.7086834733893557

In [29]:
roc_knn = roc_auc_score(df_scores['true'],df_scores['score_knn'])
roc_knn

0.6351540616246498

In [30]:
roc_tr = roc_auc_score(df_scores['true'],df_scores['score_tree'])
roc_tr

0.6919267707082833

Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ? 

In [31]:
from sklearn.metrics import precision_recall_curve

In [49]:
clf_names = df_scores.columns[1:]

In [32]:
pd_pr_log = precision_recall_curve(df_scores['true'],df_scores['score_logreg'])

In [44]:
type(pd_pr_log)

tuple

In [48]:
pd_pr_log[0][pd_pr_log[1] >=0.7].max()

0.6302521008403361

In [50]:
pr_scores = []
for clf in clf_names:
  pd_pr = precision_recall_curve(df_scores['true'], df_scores[clf])
  pr_scores.append(pd_pr[0][pd_pr[1] >= 0.7].max())


In [51]:
pr_scores

[0.6302521008403361,
 0.6228070175438597,
 0.6065573770491803,
 0.6517857142857143]

In [55]:
np.argmax(pr_scores)

3

In [56]:
clf_names[np.argmax(pr_scores)]

'score_tree'